## Névterek és érvényességi kör (namespaces and variable scope)

In [ ]:
# Egy függvény belsejében definiált változók kifelé nem látszanak.

a = 0

def f(x):
    a = x + 1
    b = a + 1
    return b


print(a)
print(f(10))
print(a)

print(b)

In [ ]:
a = 0

def f(x):
    y = x + 1
    
    def g(z):
        return z + 100
    
    b = y + 1
    return g(b)


print(f(1))
print(a)

In [ ]:
a = 0

def f(a):
    a = a + 1
    
    def g(a):
        return a + 100
    
    a = a + 1
    return g(a)


print(f(1))
print(a)

In [ ]:
a = 0


def f(x):
    a += 1
    return x + 1

print(a)
print(f(0))
print(a)

In [ ]:
def outer(x):
    a = 1
    
    def inner(y):
        a = 2
        return y
    
    result = inner(x)
    return a

print(outer(0))

A fentiekben körbejártuk a névtér (namespace) és az érvényességi kör (scope) fogalmát. Aki kíváncsi a mélyebb részletekre, annak ajánlom a [RealPython](https://realpython.com/python-namespaces-scope/) megfelelő tutorialját.

Pythonban lehetséges a globális névtérben definiált változók értékének megváltoztatása a lokális névtérben. Ekkor nem a lokális névtérben keres az interpreter egy `a`-t, hanem a globálisban. Egy függvény belsejében definiált függvény is tudja módosítani a külső függvényben definiált lokális változót, ha nagyon muszáj. Erre szolgálnak a `global` és `nonlocal` kulcsszavak, azonban egyáltalán nem baj, ha ezeket a lehetőségeket **soha nem használjuk**.

Csak azért, mert egy nyelv megenged bizonyos lehetőségeket, az nem azt jelenti, hogy azokat mind-mind jó ötlet használni. :)

In [1]:
class Circle:
    def __init__(self, radius):
        self.radius = radius

In [ ]:
c1 = Circle(1)
c2 = Circle(2)
c3 = Circle(1)

circles = {c1, c2, c3}

len(circles)

In [ ]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    def __repr__(self):
        return f"Circle({self.radius})"

In [ ]:
c1 = Circle(1)
c2 = Circle(2)
c3 = Circle(1)

circles = {c1, c2, c3}

for circle in circles:
    print(circle)

    
len(circles)